In [13]:
import os

In [14]:
pwd

'c:\\Users\\slkpd\\Name-Entity-Recognition-Using-BERT\\research'

In [15]:
os.chdir("../")

In [16]:
pwd

'c:\\Users\\slkpd\\Name-Entity-Recognition-Using-BERT'

In [17]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [18]:
from src.nerBERT.constants import *
from src.nerBERT.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [20]:
import os
from src.nerBERT.logging import logger
from transformers import BertTokenizerFast
from datasets import load_dataset, load_from_disk

c:\Users\slkpd\anaconda3\envs\NER_BERT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = BertTokenizerFast.from_pretrained(config.tokenizer_name)

    def tokenize_and_align_labels(self,examples, label_all_tokens=True):
        tokenized_inputs = self.tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
        labels = []
        for i, label in enumerate(examples["ner_tags"]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            # word_ids() => Return a list mapping the tokens
            # to their actual word in the initial sentence.
            # It Returns a list indicating the word corresponding to each token.
            previous_word_idx = None
            label_ids = []
            # Special tokens like `` and `<\s>` are originally mapped to None
            # We need to set the label to -100 so they are automatically ignored in the loss function.
            for word_idx in word_ids:
                if word_idx is None:
                   # set –100 as the label for these special tokens
                   label_ids.append(-100)
                # For the other tokens in a word, we set the label to either the current label or -100, depending on
                # the label_all_tokens flag.
                elif word_idx != previous_word_idx:
                   # if current word_idx is != prev then its the most regular case
                   # and add the corresponding token
                   label_ids.append(label[word_idx])
                else:
                   # to take care of sub-words which have the same word_idx
                   # set -100 as well for them, but only if label_all_tokens == False
                   label_ids.append(label[word_idx] if label_all_tokens else -100)
                   # mask the subword representations after the first subword

                previous_word_idx = word_idx
            labels.append(label_ids)
        tokenized_inputs["labels"] = labels
        return tokenized_inputs
    

    def convert(self):
        dataset = load_from_disk(self.config.data_path)
        dataset_pt = dataset.map(self.tokenize_and_align_labels, batched = True)
        dataset_pt.save_to_disk(os.path.join(self.config.root_dir,"dataset_transformation"))

In [22]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-10-29 18:28:43,063: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-29 18:28:43,085: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-29 18:28:43,085: INFO: common: created directory at: artifacts]
[2023-10-29 18:28:43,085: INFO: common: created directory at: artifacts/data_transformation]


c:\Users\slkpd\anaconda3\envs\NER_BERT\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\slkpd\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Saving the dataset (1/1 shards): 100%|██████████| 3453/3453 [00:00<00:00, 182111.10 examples/s]
